In [4]:
import nltk
import pandas as pd
import re

In [13]:
test_tweets = pd.read_csv("test.csv")
tweets = pd.read_csv("train.csv", index_col="Id")

In [18]:
tweets.head()

,Category,Tweet
Id,,
635769805279248384,negative,Not Available
635930169241374720,neutral,IOS 9 App Transport Security. Mm need to check...
635950258682523648,neutral,"Mar if you have an iOS device, you should down..."
636030803433009153,negative,@jimmie_vanagon my phone does not run on lates...
636100906224848896,positive,Not sure how to start your publication on iOS?...


In [22]:
tweets.describe()

,Category,Tweet
count,5970,5970
unique,4,5418
top,positive,Not Available
freq,2888,548


In [27]:
tweets["Category"].value_counts()

positive    2888
neutral     2125
negative     956
Tweet          1
Name: Category, dtype: int64